In [66]:
import pandas as pd
from datetime import datetime
import boto3
from getpass import getpass
import os
from tqdm import tqdm

1.1. Faça o download do arquivo `crime.csv`.

In [101]:
crime_df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/crime.csv')
crime_df.to_csv('crime.csv', sep=',', index=False)

1.2. Leia os dados em um DataFrame Pandas e crie a coluna `reference_date`.

In [103]:
crime_df = pd.read_csv('crime.csv', sep=',')

crime_df['reference_date'] = crime_df['Date'].apply(lambda Date: datetime.strptime(Date.split(' ')[0], '%m/%d/%Y').strftime('%Y-%m-%d'))

crime_df.head()

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Latitude,Longitude,reference_date
0,0,9446824,HX100141,01/01/2014 02:00:00 AM,0000X W ILLINOIS ST,0460,BATTERY,SIMPLE,STREET,False,False,1831,18,42.0,8.0,08B,41.890828,-87.630235,2014-01-01
1,1,9446748,HX100020,01/01/2014 12:00:00 AM,006XX N DEARBORN ST,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,False,1832,18,42.0,8.0,06,41.893542,-87.629702,2014-01-01
2,2,9446758,HX100030,01/01/2014 12:30:00 AM,052XX W RACE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,1523,15,28.0,25.0,14,41.890046,-87.756333,2014-01-01
3,3,9446760,HX100027,01/01/2014 12:30:00 AM,053XX W WELLINGTON AVE,0460,BATTERY,SIMPLE,APARTMENT,True,False,2514,25,31.0,19.0,08B,41.935011,-87.759739,2014-01-01
4,4,9446764,HX100054,01/01/2014 12:10:00 AM,014XX W LEXINGTON ST,0460,BATTERY,SIMPLE,STREET,False,False,1231,12,25.0,28.0,08B,41.872509,-87.663069,2014-01-01


1.3. Persista o dado no DataFrame em um arquivo csv e em arquivos parquet, estes últimos comprimidos e particionados pela coluna `reference_date`.

In [104]:
crime_df.to_csv('crime_reference_date.csv', sep=',', index=False)

In [106]:
crime_df.to_parquet(path='./parquet_file', engine='pyarrow', compression='gzip', partition_cols='reference_date')

1.4. Crie os recursos no AWS S3 e AWS IAM e persista os dados em seus respectivos *buckets*.

In [107]:
chave_acesso = getpass()

In [108]:
chave_secreta = getpass()

In [109]:
cliente = boto3.client('s3', aws_access_key_id=chave_acesso, aws_secret_access_key=chave_secreta)

In [110]:
BUCKET_CSV = 'm41-csv'

cliente.upload_file('crime_reference_date.csv', BUCKET_CSV, 'crime.csv')

In [111]:
BUCKET_PARQUET = 'm41-parquet'
LOCAL_BASE_PATH = './parquet_file'

parquet_file = list()
for root, dirs, files in os.walk(LOCAL_BASE_PATH):
    for file in files:
        local_path = os.path.join(root, file)
        bucket_path = os.path.relpath(local_path, LOCAL_BASE_PATH).replace('\\', '/')
        parquet_file.append((local_path, bucket_path))

for local_path, bucket_path in tqdm(parquet_file, desc='Enviando arquivos para o S3'):
    cliente.upload_file(local_path, BUCKET_PARQUET, bucket_path)


print('Upload completo')

Enviando arquivos para o S3: 100%|██████████| 365/365 [00:18<00:00, 19.44it/s]

Upload completo


1.5. Crie os recursos no AWS Athena e execute as consultas SQL da aula.

**Query 1 CSV:**

```sql
SELECT "location_description", count(1) AS "amount" FROM crime_csv
GROUP BY 1
ORDER BY 2 DESC;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_41/print/Query%201%20CSV.png)


**Query 1 PARQUET:**

```sql
SELECT "location_description", count(1) AS "amount" FROM crime_parquet
GROUP BY 1
ORDER BY 2 DESC;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_41/print/Query%201%20PARQUET.png)


- **CSV**: 47,60 MB  
- **Parquet**: 693,04 KB

O arquivo Parquet foi aproximadamente **68 vezes menor** que o arquivo CSV.

___

**Query 2 CSV:**

```sql
SELECT * FROM crime_csv
WHERE CAST(reference_date as DATE) 
    BETWEEN DATE '2014-12-01' AND DATE '2014-12-31';
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_41/print/Query%202%20CSV.png)


**Query 2 PARQUET:**

```sql
SELECT * FROM crime_parquet
WHERE CAST(reference_date as DATE) 
    BETWEEN DATE '2014-12-01' AND DATE '2014-12-31';
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_41/print/Query%202%20PARQUET.png)


- **CSV**: 47,60 MB  
- **Parquet**: 979.41 KB

O arquivo Parquet foi aproximadamente **48 vezes menor** que o arquivo CSV.

___

**Query 3 CSV:**

```sql
SELECT "location_description", count(1) AS "amount" FROM crime_csv
WHERE CAST(reference_date as DATE) 
    BETWEEN DATE '2014-12-01' AND DATE '2014-12-31'
GROUP BY 1
ORDER BY 2 DESC;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_41/print/Query%203%20CSV.png)


**Query 3 PARQUET:**

```sql
SELECT "location_description", count(1) AS "amount" FROM crime_parquet
WHERE CAST(reference_date as DATE) 
    BETWEEN DATE '2014-12-01' AND DATE '2014-12-31'
GROUP BY 1
ORDER BY 2 DESC;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_41/print/Query%203%20PARQUET.png)


- **CSV**: 47,60 MB  
- **Parquet**: 35.36 KB

O arquivo Parquet foi aproximadamente **1.346 vezes menor** que o arquivo CSV.

___